In [25]:
import subprocess
import os
from duckduckgo_search import DDGS

gem_path= r"C:\Users\amina\AppData\Roaming\npm\gemini.cmd"
# astronomy_keywords=[  "planet", "sun", "star", "galaxy", "black hole", "universe", "solar system",
#     "space", "orbit", "moon", "comet", "meteor", "astronaut", "NASA", "cosmos",
#     "Milky Way", "gravity", "big bang", "telescope", "astronomy", "nebula", "dark matter","aliens","Mars",
#     "Jupiter","Neptune","Mercure","Earth","Saturn"
# ]

# def is_astro_question(question):
#     q_lower= question.lower()
#     return any(keyword in q_lower for keyword in astronomy_keywords)

def web_search(query, max_results=3 ):
    with DDGS() as ddgs:
        results= ddgs.text(query, max_results=max_results)
        return [r["body"] for r in results]

def ask_gemini(question, web_contxt=""):
    try:
        prompt = f"""I want you to act as a astronomy expert. I will provide some questions about astronomy concepts , and it will be your job to explain them in
            easy-to-understand terms for children as old as 6 years old. 
            This could include providing easy and understandable examples, demonstrating various techniques with
                visuals or suggesting online resources for further study
                
                Here is some informations from web search:
                {web_contxt}
                Question: {question}
                Answer:"""
        result= subprocess.run(
            [gem_path,"-p",question],
            capture_output=True,
            text=True
        )
        return result.stdout.strip()
    except FileNotFoundError:
        return "gemini cli not found"
    except Exception as e:
        return f"an error occured:{str(e)}"

def astro_teacher(question):
    # if not is_astro_question(question):
    #     return "I can't answer this question"
    web_contxt= web_search(question)
    response = ask_gemini(question)
    if response:
        return response + "\n\nWould you like to know more about this topic?"
    else:
        return "Hmm, I couldn't get an answer right now. Please try again later."

# === Run interaction ===
if __name__ == "__main__":
    print("Welcome to AstroTeacher! Ask me anything about astronomy. (Type 'exit' to quit)\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit","quit"]:
            print("AstroTeacher: Goodbye! Keep looking at the stars!")
            break
        response= astro_teacher(user_input)
        print("AstroTeacher:", response)
        

Welcome to AstroTeacher! Ask me anything about astronomy. (Type 'exit' to quit)



You:  How long does it take to go to Mars


C:\Users\amina\AppData\Local\Temp\ipykernel_228\3499167807.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


AstroTeacher: The journey to Mars typically takes about 7 to 9 months. The exact duration depends on several factors, including the alignment of Earth and Mars, the chosen trajectory of the spacecraft, and the propulsion technology used.

Would you like to know more about this topic?


You:  Can I go to Mars even if Im 7 years old


C:\Users\amina\AppData\Local\Temp\ipykernel_228\3499167807.py:17: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


AstroTeacher: That's a fantastic question! It shows you have a big sense of adventure.

Right now, you can't go to Mars, and in fact, no person has ever been to Mars yet. Only very tough robots have made the journey.

Astronauts have to be adults because the trip is very hard on the human body. They also need many years of special training and education after they finish college.

But here's the exciting part: by the time you're old enough to become an astronaut, we will be much, much closer to sending people to Mars. If you want to be one of them, you can start preparing right now by:

*   **Studying hard in school**, especially in science and math.
*   **Staying healthy and active.**
*   **Always being curious** and learning as much as you can about space.

So keep that dream alive! Maybe you'll be one of the first humans to walk on the Red Planet.

Would you like to know more about this topic?


You:  exit


AstroTeacher: Goodbye! Keep looking at the stars!
